## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Modificaciones para el proceso de clusterización

---

En esta libreta se realizan las transformaciones sobre los datasets del escenario considerando su uso para los algoritmos de clusterización.

---

In [14]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [15]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_c.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_c.csv", sep=";")
display(df_tarjetas.head(1))

df_integrado = pd.read_csv("../../data/processed/datos_integrados_c.csv", sep=";")
display(df_integrado.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila_y,estado_civil_ok,estado_cliente_ok,genero_ok,nivel_educativo_ok,nivel_tarjeta_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,SECUNDARIO_COMPLETO,Blue,111.0,3.0,0.0,True,True,True,True,True


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [16]:
# Se establece qué columnas se eliminan

col_eliminar = ['nivel_tarjeta','completitud_fila_x','situacion_vivienda_ok','objetivo_credito_ok','estado_credito_ok','falta_pago_ok','completitud_fila_y','estado_civil_ok','estado_cliente_ok','genero_ok','nivel_educativo_ok','nivel_tarjeta_ok']

# Se ejecuta la operación

df_integrado.drop(col_eliminar, inplace=True, axis=1)

In [17]:
print("Vista del dataset integrado:")
display(df_integrado.head(1))

Vista del dataset integrado:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,falta_pago,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,N,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0


Limpieza de los datos (filtrado a nivel de filas)

In [18]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_integrado.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_integrado = pd.concat([df_integrado, regla_pct_ingresos], axis=1)
df_integrado.head(5)

# Cantidad de filas que no cumplen la regla 1
cantidad_err_regla_1 = (df_integrado["regla_pct_ingresos"] == "err").sum()
print(f"Cantidad de filas que no cumplen la regla 1: {cantidad_err_regla_1}")

Cantidad de filas que no cumplen la regla 1: 15


In [19]:
# Función Regla 2 (Se verifica la duración mínima permitida y se verifica si el porcentaje de ingresos excede el 60%)
def calcular_regla_2(row):
    duracion_credito = row.duracion_credito
    pct_ingreso = row.pct_ingreso
    situacion_vivienda = row.situacion_vivienda
    
    # Duración mínima permitida
    duracion_minima = df_integrado['duracion_credito'].min()
    
    # Se verifica si la duración del crédito es la mínima permitida
    if duracion_credito == duracion_minima:
        # Se verifica si el porcentaje de ingresos excede el 60%
        if pct_ingreso > 0.6:
            # Si no es propietario de su vivienda, es un error
            if situacion_vivienda != 'PROPIA':
                return 'err'
    return 'ok'

# Se aplica la función para todos los elementos del dataset
regla_duracion_ingresos = df_integrado.apply(lambda row: calcular_regla_2(row), axis=1).rename("regla_duracion_ingresos")

# Se unen los resultados al dataset inicial
df_integrado = pd.concat([df_integrado, regla_duracion_ingresos], axis=1)

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla 2:\n")
display(df_integrado[df_integrado.regla_duracion_ingresos == 'err'].head())

# Se verifica la cantidad de elementos
aux_regla_2 = df_integrado[df_integrado.regla_duracion_ingresos == 'err']
print(f"Cantidad de filas que no cumplen la regla 2: {aux_regla_2.shape[0]}")

Se visualizan las tuplas que no cumplen con la regla 2:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos,regla_duracion_ingresos
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,...,CASADO,ACTIVO,2244.0,M,19802.0,UNIVERSITARIO_INCOMPLETO,58.0,4.0,err,err
541,709186983.0,23,20000,2,1.0,ALQUILER,32900,EDUCACIÓN,0.61,16.00,...,CASADO,ACTIVO,1536.0,M,8714.0,UNIVERSITARIO_INCOMPLETO,38.0,4.0,ok,err
1646,711399408.0,24,29100,2,8.0,HIPOTECA,46000,PERSONAL,0.63,12.99,...,DIVORCIADO,PASIVO,2462.0,F,12373.0,UNIVERSITARIO_COMPLETO,40.0,2.0,ok,err
2573,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,...,CASADO,PASIVO,2275.0,F,1684.0,UNIVERSITARIO_INCOMPLETO,45.0,3.0,err,err
3041,713962233.0,22,30000,2,1.0,ALQUILER,48000,EDUCACIÓN,0.63,18.39,...,CASADO,ACTIVO,1877.0,F,3035.0,UNIVERSITARIO_COMPLETO,37.0,3.0,ok,err


Cantidad de filas que no cumplen la regla 2: 7


In [20]:
# Cantidad de filas antes de aplicar cualquier filtro
total_filas_inicial = df_integrado.shape[0]
print(f"Filas antes del filtro: {total_filas_inicial}")

# Filtro edad < 90
temp = df_integrado[df_integrado['edad'] < 90]
print(f"Filas filtradas por edad >= 90: {total_filas_inicial - temp.shape[0]}")

# Filtro por regla_pct_ingresos == 'ok'
temp_c = temp[temp['regla_pct_ingresos'] == 'ok']
print(f"Filas filtradas por regla_pct_ingresos != 'ok': {temp.shape[0] - temp_c.shape[0]}")

# Filtro por regla_duracion_ingresos == 'ok'
temp_a = temp_c[temp_c['regla_duracion_ingresos'] == 'ok']
print(f"Filas filtradas por regla_duracion_ingresos != 'ok': {temp_c.shape[0] - temp_a.shape[0]}")

# Filtro por antiguedad_empleado not null
temp_b = temp_a[temp_a['antiguedad_empleado'].notnull()]
print(f"Filas filtradas por antiguedad_empleado nulo: {temp_a.shape[0] - temp_b.shape[0]}")

# Filtro por antiguedad_empleado < 50
temp_d = temp_b[temp_b['antiguedad_empleado'] < 50]
print(f"Filas filtradas por antiguedad_empleado >= 50: {temp_b.shape[0] - temp_d.shape[0]}")

# Filtro por tasa_interes not null
df_integrado = temp_d[temp_d['tasa_interes'].notnull()].copy()
print(f"Filas filtradas por tasa_interes nulo: {temp_d.shape[0] - df_integrado.shape[0]}")

# Filas finales
print(f"Filas después de todos los filtros: {df_integrado.shape[0]}")


Filas antes del filtro: 10127
Filas filtradas por edad >= 90: 4
Filas filtradas por regla_pct_ingresos != 'ok': 15
Filas filtradas por regla_duracion_ingresos != 'ok': 5
Filas filtradas por antiguedad_empleado nulo: 336
Filas filtradas por antiguedad_empleado >= 50: 2
Filas filtradas por tasa_interes nulo: 887
Filas después de todos los filtros: 8878


In [21]:
# Se eliminan columnas de las reglas

df_integrado.drop('regla_pct_ingresos', inplace=True, axis=1)
df_integrado.drop('regla_duracion_ingresos', inplace=True, axis=1)
df_integrado.drop('id_cliente', inplace=True, axis=1)

In [22]:
df_integrado.shape

(8878, 20)

In [23]:
df_integrado.head(1)

,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0


#### Transformación de atributos

##### Procesamiento de valores nulos

Este proceso se realizó en el script de transformación inicial.

##### Procesamiento de atributos nominales

Debido a que los métodos de clustering no pueden trabajar con atributos no numéricos, se realiza una binarización.

In [24]:
data = pd.get_dummies(df_integrado)
data.head()

,edad,importe_solicitado,duracion_credito,antiguedad_empleado,ingresos,pct_ingreso,tasa_interes,estado_credito,antiguedad_cliente,gastos_ult_12m,...,estado_cliente_ACTIVO,estado_cliente_PASIVO,genero_F,genero_M,nivel_educativo_DESCONOCIDO,nivel_educativo_POSGRADO_COMPLETO,nivel_educativo_POSGRADO_INCOMPLETO,nivel_educativo_SECUNDARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_INCOMPLETO
0,24,11000,3,5.0,64800,0.17,5.79,0,36.0,15149.0,...,True,False,True,False,False,False,False,True,False,False
1,24,1500,2,0.0,30996,0.05,15.99,1,45.0,992.0,...,False,True,False,True,True,False,False,False,False,False
2,23,10000,2,7.0,40104,0.25,12.72,0,38.0,1447.0,...,False,True,False,True,False,True,False,False,False,False
3,25,6000,4,2.0,23198,0.26,8.00,0,24.0,3940.0,...,True,False,True,False,False,False,False,False,False,True
4,26,10000,2,0.0,50000,0.20,7.74,0,41.0,4369.0,...,True,False,True,False,False,False,False,False,False,True


Exportación del dataset

In [25]:
data.to_csv("../../data/final/datos_finales_clusterizacion.csv", sep=';', index=False)

In [26]:
print(f"Cantidad de filas del dataset integrado: {data.shape[0]}")
print(f"Cantidad de columnas del dataset integrado: {data.shape[1]}")

Cantidad de filas del dataset integrado: 8878
Cantidad de columnas del dataset integrado: 39
